In [1]:
import altair as alt
import pandas as pd

In [2]:
# Define custom theme to be applied to all plots
HEADER_LEGEND_FONT_SIZE = 30
FONTSIZE = 26
def theme():
    return {
        "config": {
            "title": {
                "dy": 1,
                "fontFamily": "SF Pro Text",
                "fontSize": 22,
                "fontWeight": 600,
                "align": "center",
                "anchor": "middle",
                "subtitleColor": "grey",
                "subtitleFontSize": 22
            },
            "view": {
                "fill": "#FCFCFC",
            },
            "mark": {
                "zindex": 0
            },
            "header": {
                "titleFontSize": HEADER_LEGEND_FONT_SIZE,
                "labelFontSize": HEADER_LEGEND_FONT_SIZE,
                "labelFontWeight": 600,
            },
            "axis": {
                "fontFamily": "SF Pro Text",
                "titleFontSize": 23,
                "titleFontWeight": 600,
                "labelFontSize": 20,
                "labelFontWeight": 400,
                "labelLimit": 1000,
                "domainWidth": 1.5,
                "domainColor": "black",
                "tickColor": "white",
                "domain": True,
                "zindex": -1
            },
            "legend": {
                "fontFamily": "SF Pro Text",
                "titleFontSize": 23,
                "titleFontWeight": 600,
                "labelFontSize": 23,
                "labelLimit": 1000,
                "strokeColor": '#F4F6F7',
                "padding": 15
            }
        }
    }

alt.themes.register("theme", theme)
alt.themes.enable("theme")

ThemeRegistry.enable('theme')

In [3]:
df = pd.read_csv('../input/manual-testing/hubmap_manual_test.csv')

df['TASK'] = 'T' + df.TASK_ID.astype(str) + '. ' + df.TASK_SHORT_DESC

df['MINUTES'] = df.SECONDS
df.MINUTES /= 60

df.SUCCESS = df.SUCCESS.apply(lambda x: 'Succeeded in 5 mins' if x == 'O' else 'Failed in 5 mins')

df.BASELINE = 4

df

TASKS_SORTED = df.sort_values(by='SECONDS', ascending=False).TASK.tolist()
TASKS_SORTED

['T1. Sign in',
 'T3. Filter datasets',
 'T4. Query by gene',
 'T6. Download a file',
 'T7. Find donor metadata of a dataset',
 'T2. Find publications',
 'T5. Filter donors',
 'T9. Bookmark a dataset',
 'T10. Identify visualization types',
 'T8. Find contributors of a dataset']

In [4]:
plot_result = alt.Chart(df).mark_bar().encode(
    alt.X('MINUTES:Q', title='Time taken (minutes)').scale(domain=[0, 5], clamp=True).axis(format='.0f', tickCount=5),
    alt.Y('TASK:N', title='Tasks', sort=TASKS_SORTED).axis(titleX=-360, zindex=1),
    alt.Color('SUCCESS:N', title='Task completion').scale(range=['#D55D00', '#56B4E9']).legend(orient='bottom')
).properties(
    title={
        'text': 'Task Completion',
        # 'subtitle': 'Data as of Jul 24, 2024',
        "dy": -10
    },
    width=400,
    height=300
)
plot_result

alt.Chart(...)

In [5]:
COLS = ['CONFIDENCE', 'SATISFACTION', 'FRUSTRATION']

plot_likert = None
for c in COLS:
    _plot = alt.Chart(df).mark_point(size=150, opacity=1, filled=True, color='black').encode(
        alt.X(f'{c}:Q', title=[f'{c.title()}', '(1=negative, 7=positive)']).scale(domain=[1, 7]).axis(tickCount=5),
        alt.Y('TASK:N', title='Tasks', sort=TASKS_SORTED).axis(None),
        # alt.Color('CONFIDENCE:N', title=None, legend=None).scale(domain=[1, 2, 3, 4, 5, 6, 7], range=['#56B4E9', '#56B4E9', '#56B4E9', 'grey', '#D55D00', '#D55D00', '#D55D00']),
    ).properties(
        title={
            'text': f'{c.title()} Level',
            # 'subtitle': 'Data as of Jul 24, 2024',
            "dy": -10
        },
        width=300,
        height=300
    )
    
    rule = alt.Chart(pd.DataFrame(data={ 'baseline': [4] })).mark_rule(color='black', strokeDash=[3, 3]).encode(
        alt.X('baseline:Q').scale(domain=[1, 7])
    )
    
    _plot = rule + _plot

    if plot_likert == None:
        plot_likert = _plot
    else:
        plot_likert |= _plot

plot_likert = plot_likert.resolve_scale(y='shared')

In [6]:
plot = alt.hconcat(plot_result, plot_likert).resolve_scale(y='shared').properties(
    title={
        'text': 'User Testing Results of HuBMAP Data Portal',
        'subtitle': 'Data as of Jul 24, 2024',
        "dy": -10,
        'fontSize': 30
    },
)
plot.save('../input/manual-testing/hubmap.png', scale_factor=16)
plot

alt.HConcatChart(...)